In [1]:
from langchain_community.llms import Ollama
from langchain.callbacks import StdOutCallbackHandler
from datasets import load_dataset
import pandas as pd

In [2]:
dataset_id = "glaiveai/glaive-function-calling-v2"
bos_token, eos_token = "<s>","</s>"
bop_token, eop_token = "[INST]","[/INST]"

In [3]:
instruct_tune_dataset = load_dataset(dataset_id,split="train")

In [4]:
def clean_data(record):
    system = record["system"] 

    """
    record["system"] = system.replace("SYSTEM:","System:")
    """
    
    chat = record["chat"]
    
    chat = (chat
            #.replace("USER:","Human:")
            .replace("ASSISTANT: <functioncall>","FUNCTION:")
            #.replace("ASSISTANT:","AI:")
            #.replace("FUNCTION RESPONSE:","Response:")
            .replace("<|endoftext|>","")
           )
    
    chat = chat.split("\n\n\n")

    record["prompt"] = chat[0]
    record["response"] = "\n".join(chat[1:])
    
    return record

In [5]:
instruct_tune_dataset = instruct_tune_dataset.filter(lambda c: c["chat"].find("\n\n\n") > 1)
instruct_tune_dataset = instruct_tune_dataset.map(clean_data)
instruct_tune_dataset

Dataset({
    features: ['chat', 'system', 'prompt', 'response'],
    num_rows: 79643
})

In [6]:
seed = 421337
sample_size = 100
instruct_tune_dataset = instruct_tune_dataset.shuffle(seed=seed).select(range(sample_size))
instruct_tune_dataset

Dataset({
    features: ['chat', 'system', 'prompt', 'response'],
    num_rows: 100
})

In [7]:
def create_prompt(record):
    
    system = record["system"]
    prompt = record["prompt"]
    response = record["response"]

    full_prompt = ""
    full_prompt += bos_token
    full_prompt += bop_token
    
    # system
    full_prompt += system
    
    # prompt
    full_prompt += "\n"
    full_prompt += prompt
    
    full_prompt += eop_token
    full_prompt += "\n"
    
    # response
    full_prompt += response
    full_prompt += eos_token
    full_prompt += "\n"
    
    return full_prompt

In [8]:
def extract_prompt(prompt):
    idx1 = prompt.index(bop_token)
    idx2 = prompt.index(eop_token) + len(eop_token)
    return prompt[idx1: idx2] + "\n"

In [9]:
sample = instruct_tune_dataset
complete_prompts = list(map(create_prompt,sample))
user_prompts = list(map(extract_prompt, complete_prompts))
sample_df = pd.DataFrame({"user_prompt":user_prompts,"expected":complete_prompts})


In [10]:
sample_df["func_expected"] = sample_df["expected"].apply(lambda prompt: prompt.find("FUNCTION:") > 0)

In [8]:
models = [
    #"mistral:v0.2",
    #"mistral-functioncalling-1-100",
    #"mistral-functioncalling-1-1000",
    #"mistral-functioncalling-1-5000",
    #"mistral-functioncalling-1-10000",
    #"mistral-functioncalling-2-5000",
    "mistral-functioncalling-2-10000",
    #"mistral-functioncalling-5-1000",
]

In [9]:
models

['mistral-functioncalling-2-10000']

In [10]:
sample_df = pd.read_csv("ollama_test.csv")

In [11]:
def generate(prompt):
    # print("Start","#"*10)
    # print(prompt)
    print("Generating...","#"*10)
    response = llm.invoke(prompt,num_predict=256)
    # print(response)
    # print("Done generating","#"*10)
    return response

In [12]:
%%time
for idx, model in enumerate(models):
    print(f"Switching to {model}")
    llm = Ollama(model=model,temperature=0)
    sample_df[f"{model}"] = sample_df["user_prompt"].apply(generate)
    sample_df[f"found_{model}"] = sample_df[f"{model}"].apply(lambda prompt: prompt.find("FUNCTION:") > 0)
    print("Done")
    sample_df.to_csv("ollama_test.csv",index=False)
    print("#"*10)

Switching to mistral-functioncalling-2-10000
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Generating... ##########
Gener

In [16]:
sample_df.loc[:,[
    "func_expected",
    "found_mistral:v0.2",
    "found_mistral-functioncalling-1-100",
    "found_mistral-functioncalling-1-1000",
    "found_mistral-functioncalling-1-5000",
    "found_mistral-functioncalling-1-10000",
    "found_mistral-functioncalling-2-5000",
    "found_mistral-functioncalling-2-10000",
    "found_mistral-functioncalling-5-1000"
]].sum()

func_expected                            81
found_mistral:v0.2                        0
found_mistral-functioncalling-1-100       0
found_mistral-functioncalling-1-1000      0
found_mistral-functioncalling-1-5000      6
found_mistral-functioncalling-1-10000    30
found_mistral-functioncalling-2-5000     51
found_mistral-functioncalling-2-10000    35
found_mistral-functioncalling-5-1000      6
dtype: int64

In [14]:
sample_df.to_csv("ollama_test.csv",index=False)

In [ ]:
2